# YOLOv5 Training Results Visualization
This notebook loads the history JSON files and generates plots without re-running training.

## Setup & Configuration

In [1]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration - adjust these to match your setup
BASE_OUTPUT_DIR = "/Users/shirn/Downloads/output"
SEEDS = [42, 123]

## Helper Functions

In [2]:
def load_history(seed, output_type="final"):
    """
    Load training history from JSON file.
    
    Args:
        seed: Random seed (42 or 123)
        output_type: "final" or "hpo" to load different history types
    
    Returns:
        dict: History with keys like 'loss', 'accuracy', 'test_loss', 'test_accuracy', etc.
    """
    if output_type == "final":
        # For final training, use output/history/
        hist_dir = os.path.join(BASE_OUTPUT_DIR, "history")
        # Find the file matching this seed - look for .json files
        hist_files = [f for f in os.listdir(hist_dir) if f.startswith(f"final_seed{seed}") and f.endswith('.json')]
        if hist_files:
            hist_path = os.path.join(hist_dir, hist_files[0])
        else:
            raise FileNotFoundError(f"No final training history found for seed {seed} in {hist_dir}")
    
    elif output_type == "hpo":
        # For HPO, use HPO/history/ (if it exists)
        hist_dir = os.path.join(BASE_OUTPUT_DIR, "HPO/history")
        # Look for .json files
        hist_files = [f for f in os.listdir(hist_dir) if f.startswith(f"seed{seed}") and f.endswith('.json')]
        if hist_files:
            hist_path = os.path.join(hist_dir, hist_files[0])
        else:
            raise FileNotFoundError(f"No HPO history found for seed {seed} in {hist_dir}")
    
    else:
        raise ValueError("output_type must be 'final' or 'hpo'")
    
    print(f"Loading history from: {hist_path}")
    with open(hist_path, 'r') as f:
        history = json.load(f)
    
    return history

In [3]:
def plot_curves(history, title, output_path=None):
    """
    Plot three separate figures: Accuracy, Loss, and Train-Test Gap.
    
    Args:
        history: Dict with keys 'loss', 'accuracy', 'val_loss', 'val_accuracy', 'test_loss', 'test_accuracy'
        title: Title for the plots
        output_path: Optional path to save the plots as PNG
    """
    epochs = range(1, len(history["loss"]) + 1)
    has_val = "val_accuracy" in history
    has_test = "test_accuracy" in history

    # Figure 1: Accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, history["accuracy"], label="train_acc", linewidth=2)
    if has_val:
        plt.plot(epochs, history["val_accuracy"], label="val_acc", linewidth=2)
    if has_test:
        plt.plot(epochs, history["test_accuracy"], label="test_acc", linewidth=2)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)
    plt.title(title + " - Accuracy", fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    if output_path:
        plt.savefig(output_path.replace('.png', '_accuracy.png'), dpi=300, bbox_inches='tight')
    plt.show()

    # Figure 2: Loss
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, history["loss"], label="train_loss", linewidth=2)
    if has_val:
        plt.plot(epochs, history["val_loss"], label="val_loss", linewidth=2)
    if has_test:
        plt.plot(epochs, history["test_loss"], label="test_loss", linewidth=2)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Cross-Entropy Loss", fontsize=12)
    plt.title(title + " - Loss", fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    if output_path:
        plt.savefig(output_path.replace('.png', '_loss.png'), dpi=300, bbox_inches='tight')
    plt.show()

    # Figure 3: Train-Test Gap
    if has_test:
        gap = np.array(history["accuracy"]) - np.array(history["test_accuracy"])
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, gap, label="train-test acc gap", linewidth=2, color='lightblue')
        plt.xlabel("Epoch", fontsize=12)
        plt.ylabel("Accuracy Gap", fontsize=12)
        plt.title(title + " - Train-Test Gap", fontsize=14, fontweight='bold')
        plt.legend(fontsize=11)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        if output_path:
            plt.savefig(output_path.replace('.png', '_gap.png'), dpi=300, bbox_inches='tight')
        plt.show()

## Training Summaries from CSV Files

In [4]:
def create_summary_table():
    """Load and display summary tables from CSV files."""
    print("\n" + "="*80)
    print("TRAINING SUMMARIES FROM CSV FILES")
    print("="*80 + "\n")
    
    # Final training summary
    final_summary_path = os.path.join(BASE_OUTPUT_DIR, "outputs/results/final_training_summary.csv")
    if os.path.exists(final_summary_path):
        print("\nFINAL TRAINING SUMMARY:")
        print("-"*80)
        final_df = pd.read_csv(final_summary_path)
        print(final_df[["seed", "total_epochs", "final_train_acc", "final_test_acc"]].to_string(index=False))
        print(f"\nAverage Test Accuracy: {final_df['final_test_acc'].mean():.4f} ± {final_df['final_test_acc'].std():.4f}")
    else:
        print(f"Final summary not found at: {final_summary_path}")
    
    # HPO summary
    hpo_summary_path = os.path.join(BASE_OUTPUT_DIR, "HPO/results/best_over_seeds.csv")
    if os.path.exists(hpo_summary_path):
        print("\n" + "="*80)
        print("HPO BEST CONFIGURATIONS:")
        print("-"*80)
        hpo_df = pd.read_csv(hpo_summary_path)
        print(hpo_df[["seed", "unfreeze_from", "dropout", "best_val_acc", "test_acc"]].to_string(index=False))
    else:
        print(f"HPO summary not found at: {hpo_summary_path}")

create_summary_table()


TRAINING SUMMARIES FROM CSV FILES

Final summary not found at: /Users/shirn/Downloads/output/outputs/results/final_training_summary.csv
HPO summary not found at: /Users/shirn/Downloads/output/HPO/results/best_over_seeds.csv


## Final Training Results - Plots

In [5]:
def plot_final_training_results():
    """Plot final training results for both seeds."""
    print("\n" + "="*80)
    print("FINAL TRAINING RESULTS - PLOTS FROM SAVED OUTPUTS")
    print("="*80 + "\n")
    
    for seed in SEEDS:
        print(f"\n{'#'*60}")
        print(f"# Plotting Final Training - SEED {seed}")
        print(f"{'#'*60}")
        
        try:
            history = load_history(seed, output_type="final")
            
            title = f"YOLOv5 Final Training (Seed {seed})"
            
            output_path = os.path.join(BASE_OUTPUT_DIR, "outputs/figures", 
                                      f"final_seed{seed}_plots.png")
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            plot_curves(history, title, output_path=output_path)
            
            # Print summary stats
            print(f"\nFinal Training Stats (Seed {seed}):")
            print(f"  Final Train Accuracy: {history['accuracy'][-1]:.4f}")
            if "test_accuracy" in history:
                print(f"  Final Test Accuracy:  {history['test_accuracy'][-1]:.4f}")
                print(f"  Train-Test Gap:       {history['accuracy'][-1] - history['test_accuracy'][-1]:.4f}")
            print(f"  Final Train Loss:     {history['loss'][-1]:.4f}")
            if "test_loss" in history:
                print(f"  Final Test Loss:      {history['test_loss'][-1]:.4f}")
            
        except Exception as e:
            print(f"Error loading history for seed {seed}: {e}")

plot_final_training_results()


FINAL TRAINING RESULTS - PLOTS FROM SAVED OUTPUTS


############################################################
# Plotting Final Training - SEED 42
############################################################
Loading history from: /Users/shirn/Downloads/output/history/final_seed42_unfreezelayer6_drop0.2.json
Error loading history for seed 42: 'RcParams' object has no attribute '_get'

############################################################
# Plotting Final Training - SEED 123
############################################################
Loading history from: /Users/shirn/Downloads/output/history/final_seed123_unfreezelayer6_drop0.2.json
Error loading history for seed 123: 'RcParams' object has no attribute '_get'
Error loading history for seed 42: 'RcParams' object has no attribute '_get'

############################################################
# Plotting Final Training - SEED 123
############################################################
Loading history from: /Users/shirn

## HPO Results - Plots (Optional)

In [6]:
def plot_hpo_results():
    """Plot HPO results for both seeds."""
    print("\n" + "="*80)
    print("HPO RESULTS - PLOTS FROM SAVED OUTPUTS")
    print("="*80 + "\n")
    
    for seed in SEEDS:
        print(f"\n{'#'*60}")
        print(f"# Plotting HPO Results - SEED {seed}")
        print(f"{'#'*60}")
        
        try:
            history = load_history(seed, output_type="hpo")
            
            title = f"YOLOv5 HPO Training (Seed {seed})"
            
            output_path = os.path.join(BASE_OUTPUT_DIR, "HPO/figures", 
                                      f"hpo_seed{seed}_plots.png")
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            plot_curves(history, title, output_path=output_path)
            
            # Print summary stats
            print(f"\nHPO Training Stats (Seed {seed}):")
            print(f"  Final Train Accuracy: {history['accuracy'][-1]:.4f}")
            if "val_accuracy" in history:
                print(f"  Final Val Accuracy:   {history['val_accuracy'][-1]:.4f}")
            if "test_accuracy" in history:
                print(f"  Final Test Accuracy:  {history['test_accuracy'][-1]:.4f}")
            print(f"  Final Train Loss:     {history['loss'][-1]:.4f}")
            
        except Exception as e:
            print(f"Error loading history for seed {seed}: {e}")

# Uncomment below to plot HPO results
# plot_hpo_results()

## Summary

In [8]:
print("\n" + "="*80)
print("PLOTTING COMPLETE")
print("="*80)

# Display all plots from final training
print("\n" + "="*80)
print("DISPLAYING FINAL TRAINING PLOTS")
print("="*80)

for seed in SEEDS:
    try:
        history = load_history(seed, output_type="final")
        title = f"YOLOv5 Final Training (Seed {seed})"
        plot_curves(history, title)
    except Exception as e:
        print(f"Error displaying plots for seed {seed}: {e}")


PLOTTING COMPLETE

DISPLAYING FINAL TRAINING PLOTS
Loading history from: /Users/shirn/Downloads/output/history/final_seed42_unfreezelayer6_drop0.2.json
Error displaying plots for seed 42: 'RcParams' object has no attribute '_get'
Loading history from: /Users/shirn/Downloads/output/history/final_seed123_unfreezelayer6_drop0.2.json
Error displaying plots for seed 123: 'RcParams' object has no attribute '_get'


## Display All Training Plots

In [ ]:
# Generate and display all final training plots
print("\n" + "="*80)
print("GENERATING FINAL TRAINING PLOTS")
print("="*80 + "\n")

for seed in SEEDS:
    try:
        print(f"\n{'='*80}")
        print(f"SEED {seed} - Final Training Plots")
        print(f"{'='*80}")
        history = load_history(seed, output_type="final")
        title = f"YOLOv5 Final Training (Seed {seed})"
        plot_curves(history, title)
        
        # Print statistics
        print(f"\n📊 Statistics for Seed {seed}:")
        print(f"  ✓ Final Train Accuracy: {history['accuracy'][-1]:.4f}")
        print(f"  ✓ Final Test Accuracy:  {history['test_accuracy'][-1]:.4f}")
        print(f"  ✓ Train-Test Gap:       {history['accuracy'][-1] - history['test_accuracy'][-1]:.4f}")
        print(f"  ✓ Final Train Loss:     {history['loss'][-1]:.4f}")
        print(f"  ✓ Final Test Loss:      {history['test_loss'][-1]:.4f}")
        
    except Exception as e:
        print(f"❌ Error processing seed {seed}: {e}")

print("\n" + "="*80)
print("✅ ALL PLOTS DISPLAYED SUCCESSFULLY")
print("="*80)